1. Display the number of votes for each post title

2. Display posts with comments created by users living in the same location as the post creator

3. How many users have never voted ?

4. Display all posts having the highest amount of comments

5. For each post, how many votes are coming from users living in Canada ? What’s their percentage of the total number of votes

6. How many hours in average, it takes to the first comment to be posted after a creation of a new post

7. Whats the most common post tag ?

8. Create a pivot table displaying how many posts were created for each year (Y axis) and each month (X axis)